# Handling images with Pytorch

## Image dataset

In [ ]:
from torchvision.datasets import ImageFolder
from torchvision import transforms

from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from torchmetrics import Precision, Recall

In [ ]:
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128, 128))
])

dataset_train = ImageFolder(
    "data/clouds_train",
    transform=train_transforms
)

## Data augmentation in Pytorch

In [ ]:
train_transforms = transforms.Compose([
  transforms.RandomHorizontalFlip(),
  transforms.RandomRotation(45),
  transforms.ToTensor(),
  transforms.Resize((128, 128))
])

dataset_train = ImageFolder(
  "data/clouds_train",
  transform=train_transforms,
)

dataloader_train = DataLoader(
  dataset_train, shuffle=True, batch_size=1
)

image, label = next(iter(dataloader_train))
image = image.squeeze().permute(1, 2, 0) 
plt.imshow(image)
plt.show()

# Convolutional neural networks

## Building convolutional networks

In [ ]:
class Net(nn.Module):
  def __init__(self, num_classes):
    super().__init__()
    self.feature_extractor = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=3, padding=1),
        nn.ELU(),
        nn.MaxPool2d(kernel_size=2),
        nn.Conv2d(32, 64, kernel_size=3, padding=1),
        nn.ELU(),
        nn.MaxPool2d(kernel_size=2),
        nn.Flatten(),
    )
    # Define classifier
    self.classifier = nn.Linear(64*16*16, num_classes)

  def forward(self, x):  
    x = self.feature_extractor(x)
    x = self.classifier(x)
    return x

# Training image classifiers

## Dataset with augmentations

In [ ]:
train_transforms = transforms.Compose([
  transforms.RandomHorizontalFlip(),
  transforms.RandomRotation(45),
  transforms.RandomAutocontrast(),
  transforms.ToTensor(),
  transforms.Resize((64, 64))
])

dataset_train = ImageFolder(
  "clouds_train",
  transform=train_transforms,
)
dataloader_train = DataLoader(
  dataset_train, shuffle=True, batch_size=16
)

## Image classifier training loop

In [ ]:

net = Net(num_classes=7)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(3):
    running_loss = 0.0

    for images, labels in dataloader_train:
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(dataloader_train)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

# Evaluating image classifier

## Multi-class model evaluation

In [ ]:
metric_precision = Precision(task="multiclass", num_classes=7, average=None)
metric_recall = Recall(task="multiclass", num_classes=7, average=None)

net.eval()
with torch.no_grad():
  for images, labels in dataloader_test:
    outputs = net(images)
    _, preds = torch.max(outputs, 1)
    metric_precision(preds, labels)
    metric_recall(preds, labels)

precision = metric_precision.compute()
recall = metric_recall.compute()
print(f"Precision: {precision}")
print(f"Recall: {recall}")

## Analyzing metrics per class

In [ ]:
metric_precision = Precision(
  task="multiclass", num_classes=7, average=None
)

net.eval()
with torch.no_grad():
  for images, labels in dataloader_test:
    outputs = net(images)
    _, preds = torch.max(outputs, 1)
    metric_precision(preds, labels)
precision = metric_precision.compute()

precision_per_class = {
  k: precision[v].item()
  for k, v 
  in dataset_test.class_to_idx.items()
}
print(precision_per_class)